<center>

---
---

# OLÉ x candl

</center>

---
---

This notebook shows how to feed `candl` [1] likelihoods into OLÉ.
It is an adaptation and generalisation of the `candl_act_class_nuts.py` and `candl_spt_class_nuts.py` examples written by Sven.
At the moment this works for the ACT likelihood, SPT support is experimental and not well tested.

*Note*: this tutorial uses `candl`, install it via `pip install candl-like`

[1] https://arxiv.org/abs/2401.13433, https://github.com/Lbalkenhol/candl

In [8]:
from jax import config
import jax.numpy as jnp
import jax

import time
import numpy as np
from functools import partial

# OLE
from OLE.theory import Theory
from OLE.likelihood import Likelihood
from OLE.sampler import EnsembleSampler, Sampler, NUTSSampler

# candl
import candl
import candl.data

# CLASS
import classy

# Initialise candl Likelihood

In [9]:
candl_like = candl.Like(candl.data.ACT_DR4_TTTEEE)

Successfully initialised candl likelihood 'ACT DR4 TT/TE/EE (Choi et al. 2020, Aiola et al. 2020)' (type: <class 'candl.likelihood.Like'>).
Data loaded from '/home/guenther/software/miniconda3/envs/OLE/lib/python3.10/site-packages/candl/data/ACT_DR4_CMB_only_v0/'.
Functional likelihood form: gaussian
--------------------------------------------------------------------------------
It will analyse the following spectra:

TT dxd       (40 bins, bin centres spanning ell = 600.3 - 4123.0)
TE dxd       (45 bins, bin centres spanning ell = 350.5 - 4123.0)
EE dxd       (45 bins, bin centres spanning ell = 350.3 - 4122.9)
TT wxw       (40 bins, bin centres spanning ell = 600.4 - 4122.3)
TE wxw       (45 bins, bin centres spanning ell = 350.5 - 4122.4)
EE wxw       (45 bins, bin centres spanning ell = 350.4 - 4122.4)
--------------------------------------------------------------------------------
A data model consisting of 1 transformations has been initialised.
The following transformations wil

# Theory Class Definition

In [10]:
class MyTheory(Theory):
    def initialize(self, **kwargs):
        super().initialize(**kwargs)   

        # input parameters of the theory
        self.requirements = ['h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'logA']

        self.cosmo = classy.Class()

        self.class_settings = {'output':'tCl,pCl,lCl,mPk',
                        'lensing':'yes',
                        'l_max_scalars': candl_like.ell_max,#7925 for ACT DR4, 3200, #  for SPT3G 2018
                        'N_ur':3.048,
                        'output_verbose':1,
                        }
        return 
    

    def compute(self, state):
        # Compute the observable for the given parameters.
        class_input = self.class_settings.copy()
        class_input['A_s'] = 1e-10*np.exp(state['parameters']['logA'][0])
        class_input['h'] = state['parameters']['h'][0]
        class_input['n_s'] = state['parameters']['n_s'][0]
        class_input['omega_b'] = state['parameters']['omega_b'][0]
        class_input['omega_cdm'] = state['parameters']['omega_cdm'][0]
        class_input['tau_reio'] = state['parameters']['tau_reio'][0]

        if state['parameters']['tau_reio'][0] < 0.01:
            class_input['tau_reio'] = 0.01

        self.cosmo.set(class_input)
        self.cosmo.compute()

        cls = self.cosmo.lensed_cl(candl_like.ell_max)

        state['quantities']['tt'] = cls['tt']
        state['quantities']['ee'] = cls['ee']
        state['quantities']['te'] = cls['te']
        state['quantities']['bb'] = cls['bb']

        return state

# Likelihood Class Definition

In [11]:
class OleCandlLikelihood(Likelihood):
    """
    OLÉ x candl wrapper
    This is a thin wrapper for candl likelihoods in OLÉ.
    """

    def initialize(self, **kwargs):
        """
        Initialisation of the likelihood, called right before sampling begins.
        Supply the requested data set via 'candl_dataset' as a path to a data set '.yaml' file.
        Note, you can pass candl shortcuts here, i.e. candl.data.ACT_DR4_TTTEEE will work, provided you have imported candl.data
        """

        print(kwargs)

        # Instantiate candl likelihood
        # By default internal priors in candl are NOT cleared, but you can override this by passing clear_candl_priors = True
        clear_priors = False
        if "clear_candl_priors" in kwargs:
            if kwargs["clear_candl_priors"]:
                clear_priors = True
        if clear_priors:
            self.candl_like = candl.Like(kwargs["candl_dataset"], priors=[])
        else:
            self.candl_like = candl.Like(kwargs["candl_dataset"])
        
        # Grab required parameters (from data model and priors)
        self.input_keys = list(np.unique(self.candl_like.required_nuisance_parameters + self.candl_like.required_prior_parameters))

        # Grab spectrum conversion helper
        self.cl2dl = (2.7255e6**2) * self.candl_like.ells * (self.candl_like.ells + 1) / (2.0 * jnp.pi)

        return

    # @partial(jax.jit, static_argnums=(0,))
    def loglike(self, state):
        """
        Compute the loglikelihood for the given parameters.
        Assumes the spectra are supplied from 0 to the correct ell_max by the theory code.
        """

        # Grab calculated spectra, convert to Dl
        Dl = {'ell': self.candl_like.ells}
        for spec_type in self.candl_like.unique_spec_types:
            Dl[spec_type] = state['quantities'][spec_type.lower()][2:] * self.cl2dl
        candl_input = {"Dl": Dl}

        # Push required parameters into dictionary for candl, convert any names
        for key in self.input_keys:
            if key == 'tau':
                candl_input['tau'] = state['parameters']['tau_reio'][0]
            else:
                candl_input[key] = state['parameters'][key][0]
        
        # Hand off to candl
        loglike = self.candl_like.log_like(candl_input)

        return jnp.array([loglike])

# Configure the Emulator and the Sampler

In [12]:
emulator_settings = {
    # the number of data points in cache before the emulator is to be trained
    'min_data_points': 80,

    # name of the cache file
    #'cache_file': './output_clang_spt_nuts/cache.pkl',
    'cache_file': './output_clang_act_nuts/cache.pkl',

    # accuracy parameters for loglike:
    'quality_threshold_constant': 1.0,
    'quality_threshold_linear': 0.1,

    # related so sampler
    'explained_variance_cutoff': 0.9999,

    # cache criteria
    'dimensionality': 7,
    'N_sigma': 5.0,

    #'compute_data_covmat': True,
    'data_covmat_directory': './act_data_covmats',

    # 'plotting_directory': './plots_sampler_clang_nuts',
    # 'testset_fraction': 0.1,
    'logfile': './output_clang_act_nuts/log.txt',

    'learning_rate': 0.1,
    'num_iters': 300,
}

likelihood_settings = {
    'candl_dataset': 'candl.data.ACT_DR4_TTTEEE',
    'clear_candl_priors': False,
}

sampler_settings = {
    # output directory
    'output_directory': './output_clang_act_nuts',
}


In [13]:
# Load the sampler
my_sampler = NUTSSampler(debug=False)

# Define cosmological parameters
my_parameters = {'h': {'prior': {'min': 0.60, 'max': 0.80}, 
                       'ref': {'mean': 0.68, 'std': 0.01},
                       'proposal': 0.01,},
                    'n_s': {'prior': {'min': 0.9, 'max': 1.1}, 'ref': {'mean': 0.965, 'std': 0.005},
                            'proposal': 0.01,}, 
                    'omega_b': {'prior': {'min': 0.02, 'max': 0.024}, 'ref': {'mean': 0.0223, 'std': 0.0003},
                                'proposal': 0.0002,}, 
                    'omega_cdm': {'prior': {'min': 0.10, 'max': 0.14}, 'ref': {'mean': 0.120, 'std': 0.002},
                                  'proposal': 0.002,}, 
                    'tau_reio': {'prior': {'min': 0.01, 'max': 0.1}, 'ref': {'mean': 0.055, 'std': 0.01},
                                 'proposal': 0.01,},
                    'logA': {'prior': {'min': 2.8, 'max': 3.3}, 'ref': {'mean': 3.1, 'std': 0.05},
                             'proposal': 0.05},
}

# Nuisance parameters are automaticailly loaded
# Helpful code for adding nuisance parameters from ACT and SPT candl likelihoods below
if "ACT DR4 TT/TE/EE" in candl_like.name:
    my_parameters['yp'] = {'prior': {'min': 0.9, 'max': 1.1}, 'ref': {'mean': 1.0, 'std': 0.0001}, 'proposal': 0.0001}
elif "SPT-3G 2018 TT/TE/EE" in candl_like.name:
    for par_name in candl_like.required_nuisance_parameters:
        for prior in candl_like.priors:
            if par_name in prior.par_names and par_name not in my_parameters:
                
                # Grab the prior central value and width
                prior_central_val = float(prior.central_value[prior.par_names.index(par_name)])
                prior_width = np.sqrt(np.diag(prior.prior_covariance)[prior.par_names.index(par_name)])
                
                # Add to Cobaya's parameters
                my_parameters[par_name] = {'prior':
                                                {'min': prior_central_val-10*prior_width,
                                                    'max': prior_central_val+10*prior_width},
                                                'ref': {'mean': prior_central_val, 'std': prior_width},
                                               'proposal': prior_width}


# Run OLÉ!

In [14]:
start = time.time()

my_sampler.initialize(theory=MyTheory(),# Theory code defined at the start
                      likelihood=OleCandlLikelihood(),# Likelihood wrapper defined at the start
                      parameters=my_parameters,# Parameters to sample
                      #   covmat = None,
                      emulator_settings = emulator_settings,
                      sampler_settings = sampler_settings,
                      likelihood_settings = likelihood_settings,
                      )

# Note the total run steps are   (nsteps * nwalkers * MPI_size)
n_steps = 10
my_sampler.run_mcmc(n_steps)

end = time.time()
print("Time elapsed: ", end - start)

ImportError: cannot import name 'candl_likelihood' from 'OLE.likelihoods.cosmo' (unknown location)